# 20 years of LEO
In this example we will consider the tracked population of objects orbiting in Low Earth Orbit and evolve it for twenty years while counting collisions and removing decayed objects.
We start, as always, with some imports:

In [1]:
# core imports
import pykep as pk
import numpy as np
import pickle as pkl

import cascade as csc
from copy import deepcopy
from tqdm.notebook import tqdm
import heyoka as hy

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot as plt
%matplotlib inline

## Loading the initial LEO population
To define the initial LEO population as tracked by the [US Space Surveillance Network (SSN)](https://en.wikipedia.org/wiki/United_States_Space_Surveillance_Network). The necessary steps are described by the code described in [The current LEO population](<./../utilities/leo_population.ipynb>)

The file needed is:
* **leo_population.pk** - created by [The current LEO population](<./../utilities/leo_population.ipynb>) notebook.

In [2]:
# r is in km and v in km/s
with open("data/leo_population.pk", "rb") as file:
    r_ic,v_ic,to_satcat_index,satcat = pkl.load(file)

* **r**: contains the initial position of all satellites to be simulated (SI units)
* **v**: contains the initial velocity of all satellites to be simulated (SI units)
* **radius**: contains all the estimated radii for the various objects (in meters)
* **to_satcat_index**: contains the indexes in the satcat of the corresponding r,v,radius entry
* **satcat**: the database created of all tracked objects

The object described by the entry ```satcat[to_satcat_index[j]]``` has initial position ```r[j]``` and initial velocity ```v[j]```. Let us inspect one entry

In [3]:
print("Dictionary entry: ", satcat[to_satcat_index[3685]])
print("Position (km): ", r_ic[3685])
print("Velocity (km/s): ", v_ic[3685])

Dictionary entry:  {'CCSDS_OMM_VERS': '2.0', 'COMMENT': 'GENERATED VIA SPACE-TRACK.ORG API', 'CREATION_DATE': '2022-02-03T04:23:25', 'ORIGINATOR': '18 SPCS', 'OBJECT_NAME': 'SL-14 DEB', 'OBJECT_ID': '1978-100F', 'CENTER_NAME': 'EARTH', 'REF_FRAME': 'TEME', 'TIME_SYSTEM': 'UTC', 'MEAN_ELEMENT_THEORY': 'SGP4', 'EPOCH': '2022-02-02T22:42:05.237280', 'MEAN_MOTION': '12.56769207', 'ECCENTRICITY': '0.00286110', 'INCLINATION': '82.3526', 'RA_OF_ASC_NODE': '268.9057', 'ARG_OF_PERICENTER': '297.9015', 'MEAN_ANOMALY': '75.7071', 'EPHEMERIS_TYPE': '0', 'CLASSIFICATION_TYPE': 'U', 'NORAD_CAT_ID': '19133', 'ELEMENT_SET_NO': '999', 'REV_AT_EPOCH': '93528', 'BSTAR': '0.02113600000000', 'MEAN_MOTION_DOT': '0.00003527', 'MEAN_MOTION_DDOT': '0.0000000000000', 'SEMIMAJOR_AXIS': '7814.445', 'PERIOD': '114.580', 'APOAPSIS': '1458.668', 'PERIAPSIS': '1413.952', 'OBJECT_TYPE': 'DEBRIS', 'RCS_SIZE': 'MEDIUM', 'COUNTRY_CODE': 'CIS', 'LAUNCH_DATE': '1978-10-26', 'SITE': 'PKMTR', 'DECAY_DATE': None, 'FILE': '329

on top of all the info distributed from the [US Space Surveillance Network (SSN)](https://en.wikipedia.org/wiki/United_States_Space_Surveillance_Network) we have added to the satcat (see [The current LEO population](<./../utilities/leo_population.ipynb>) notebook)an estimate of the object radius which we will need to define the various collisional radii. As we will be using a dynamics that also models the atmospheric drag via the BSTAR coefficient, we create the array to be used as parameter when instantiating a :class:`~cascade.sim`.

In [4]:
# Array containing the BSTAR coefficient in the SI units used
BSTARS = []
RADIUS = []
for idx in to_satcat_index:
    BSTARS.append(float(satcat[idx]["BSTAR"]))
    RADIUS.append(float(satcat[idx]["RADIUS"]))
# We transform the BSTAR in SI units
BSTARS = np.array(BSTARS) / pk.EARTH_RADIUS
RADIUS = np.array(RADIUS)
# .. and remove negative BSTARS (this can happen for objects that where performing orbital manouvres during the tle definition) setting the value to zero in those occasions.
BSTARS[BSTARS<0] = 0.
# We also transform r_ic and v_ic in SI
r_ic = r_ic*1000
v_ic = v_ic*1000

# Building the dynamical system to integrate
The dynamics in the LEO environment is dominated by drag and gravitational effects. The effect of the Moon gravity, Sun gravity and solar radiation pressure are not considered, even if they would not add much complexity in this simulation. We make use of :class:`~cascade.dynamics.simple_earth` defining the analytical expressions for such a dynamics.

In [5]:
dyn =  csc.dynamics.simple_earth(J2=True, C22S22=True,sun=False,moon=False,SRP=False,drag=True)


and we inspect it visually to check that all is defined as expected (the analytical expression are long and complex, but this will not be an issue for heyoka).

In [6]:
dyn

[(x, vx),
 (y, vy),
 (z, vz),
 (vx,
  ((((((-398600440779972.44 * x) * pow((x**2 + y**2 + z**2), -1.5000000000000000)) + (((-1.7555131752869966e+19 / (2.0000000000000000 * sqrt((x**2 + y**2 + z**2)))) * x) * ((3.0000000000000000 / (x**2 + y**2 + z**2)**2) - ((15.000000000000000 * z**2) / ((x**2 + y**2 + z**2)**2 * (x**2 + y**2 + z**2)))))) + ((((((7.6591108648176013e+17 / (2.0000000000000000 * pow((x**2 + y**2 + z**2), 3.5000000000000000))) * ((x * cos((4.8949608921188084 + (7.2921158548340406e-05 * t)))) + (y * sin((4.8949608921188084 + (7.2921158548340406e-05 * t)))))) * (((-x * sin((4.8949608921188084 + (7.2921158548340406e-05 * t)))) + (y * cos((4.8949608921188084 + (7.2921158548340406e-05 * t)))))**2 - ((x * cos((4.8949608921188084 + (7.2921158548340406e-05 * t)))) + (y * sin((4.8949608921188084 + (7.2921158548340406e-05 * t)))))**2)) + ((1.5318221729635203e+17 / pow((x**2 + y**2 + z**2), 2.5000000000000000)) * ((x * cos((4.8949608921188084 + (7.2921158548340406e-05 * t)))) + (y *

# We setup the simulation

the global cascade logger is here informed of the level of information we want to be reported to screen during the simulation.

In [7]:
csc.set_logger_level_trace()

We now define the radius that will be used to check for decayed objects. We will assume that once the position of some object is below 150km altitude, the object can be considered as decayed.

In [8]:
decay_radius = pk.EARTH_RADIUS+150000.

In [9]:
def remove_particle(idx, r_ic, v_ic, BSTARS,to_satcat, RADIUS):
    r_ic = np.delete(r_ic, idx, axis=0)
    BSTARS = np.delete(BSTARS, idx, axis=0)
    v_ic = np.delete(v_ic, idx, axis=0)
    to_satcat = np.delete(to_satcat, idx, axis=0)
    RADIUS = np.delete(RADIUS, idx, axis=0)
    return r_ic, v_ic, BSTARS, to_satcat, RADIUS

inside_the_radius = np.where(np.linalg.norm(r_ic,axis=1) < decay_radius)[0]
print("Removing ", len(inside_the_radius), " orbiting objects:")
for idx in inside_the_radius:
    print(satcat[to_satcat_index[idx]]["OBJECT_NAME"], "-", satcat[to_satcat_index[idx]]["OBJECT_ID"])
r_ic, v_ic, BSTARS,to_satcat, RADIUS = remove_particle(inside_the_radius, r_ic, v_ic, BSTARS,to_satcat_index, RADIUS)

Removing  22  orbiting objects:
LEMUR 2 ROCKETJONAH - 2017-071E
ISARA - 2017-071P
FREGAT DEB - 2011-037EM
STARLINK-1684 - 2020-070H
COSMOS 1408 DEB - 1982-092Z
COSMOS 1408 DEB - 1982-092AK
COSMOS 1408 DEB - 1982-092ES
COSMOS 1408 DEB - 1982-092FK
COSMOS 1408 DEB - 1982-092FY
COSMOS 1408 DEB - 1982-092GU
COSMOS 1408 DEB - 1982-092NA
COSMOS 1408 DEB - 1982-092PV
COSMOS 1408 DEB - 1982-092PW
COSMOS 1408 DEB - 1982-092RM
COSMOS 1408 DEB - 1982-092ACG
COSMOS 1408 DEB - 1982-092AQC
COSMOS 1408 DEB - 1982-092ARK
COSMOS 1408 DEB - 1982-092AXA
COSMOS 1408 DEB - 1982-092AXD
COSMOS 1408 DEB - 1982-092BDB
COSMOS 1408 DEB - 1982-092BFU
COSMOS 1408 DEB - 1982-092BKD


We can now instantiate the cascade simulation. This will trigger the JIT compilation of the numerical integrators with the selected dynamics with event detection and thus take a few seconds. Note that this is to be paid only once and as far as the dynamics is kept the same simulations can be made reusing the same simulation object.

In [10]:
# Prepare the data in the shape expected by the simulation object.
ic_state = np.hstack([r_ic, v_ic, RADIUS.reshape((r_ic.shape[0], 1))])
BSTARS = BSTARS.reshape((r_ic.shape[0], 1))
sim = csc.sim(ic_state, 0.23, dyn=dyn, pars=BSTARS, c_radius=decay_radius)

[2023-01-31 14:30:10.103] [cascade] [trace] Integrators setup time: 11.150963585s
[2023-01-31 14:30:10.319] [cascade] [trace] JIT functions setup time: 0.215017517s


# We run the simulation

In [16]:
# new_state  =deepcopy(ic_state)
# new_pars  =deepcopy(pars)
# new_to_satcat  =deepcopy(to_satcat)

rng = np.random


In [16]:
final_t = 365.25 * pk.DAY2SEC

pbar = tqdm(total=final_t)

while sim.time < final_t:
    orig_time = sim.time
    
    oc = sim.step()
    
    pbar.update(sim.time - orig_time)
   
    if oc == csc.outcome.collision:
        # TODO different code needed for crash
        # on Earth here.
        pi, pj = sim.interrupt_info
        
        print("Collision detected, re-initing particles {} and {}".format(pi, pj))
        
        for idx in [pi, pj]:
            a = rng.uniform(1.02*Re, 1.3*Re)
            e = rng.uniform(0, 0.02)
            inc = rng.uniform(0, 0.05)
            om = rng.uniform(0, 2*np.pi)
            Om = rng.uniform(0, 2*np.pi)
            nu = rng.uniform(0, 2*np.pi)
            size = rng.uniform(0.01, 0.1)

            r, v = pk.par2ic([a, e, inc, om, Om, nu], pk.MU_EARTH)

            sim.state[idx,0:3] = r
            sim.state[idx,3:6] = v
            sim.state[idx,6] = size
    elif oc == csc.outcome.reentry:
        pi = sim.interrupt_info
        # We log on screen 
        print(satcat[to_satcat[pi]]["OBJECT_NAME"].strip() + ", " + satcat[to_satcat[pi]]["OBJECT_ID"].strip() + ", ", sim.time*pk.SEC2DAY, "REMOVED")
        # We remove the re-entered object and restart the simulation
        new_state = np.delete(sim.state,pi,axis=0)
        new_pars = np.delete(sim.pars,pi,axis=0)
        
        sim.set_new_state(new_state)
        sim.pars[:] = new_pars
        
        # new_r_ic = np.vstack((sim.x,sim.y,sim.z)).transpose()
        # new_v_ic = np.vstack((sim.vx,sim.vy,sim.vz)).transpose()
        # new_r_ic, new_v_ic, new_BSTARS,new_to_satcat, new_c_radius = remove_particle(pi, new_r_ic, new_v_ic, new_BSTARS,new_to_satcat, new_c_radius)
        # sim.set_new_state(new_r_ic[:,0],new_r_ic[:,1],new_r_ic[:,2],new_v_ic[:,0],new_v_ic[:,1],new_v_ic[:,2],new_c_radius, pars=[new_BSTARS])
pbar.close()
del pbar

  0%|          | 0/31557600.0 [00:00<?, ?it/s]

COSMOS 1408 DEB, 1982-092FH,  0.0011018046992662697 REMOVED
COSMOS 1408 DEB, 1982-092GL,  0.0041028438559408695 REMOVED
SL-4 R/B, 2006-061B,  0.005706346736241129 REMOVED
FREGAT DEB, 2011-037EQ,  0.007267201427187295 REMOVED
STARLINK-2025, 2021-009BM,  0.01815315899057631 REMOVED
COSMOS 1408 DEB, 1982-092ADW,  0.02952830918037765 REMOVED
COSMOS 2241, 1993-022A,  0.04331453508044084 REMOVED
OBJECT K, 2021-002K,  0.060801872464038606 REMOVED
RESURS O1 DEB, 1994-074BA,  0.07344824298679883 REMOVED
COSMOS 1408 DEB, 1982-092TY,  0.07895270149013031 REMOVED
FREGAT DEB, 2011-037NN,  0.07895517418611112 REMOVED
COSMOS 1408 DEB, 1982-092AHA,  0.11577314956600732 REMOVED
STARLINK-1906, 2020-074AC,  0.1579487707670265 REMOVED
OBJECT B, 2021-117B,  0.5273644864028021 REMOVED
COSMOS 2251 DEB, 1993-036AEX,  0.6732504658829894 REMOVED
COSMOS 1408 DEB, 1982-092FX,  1.1283422206984879 REMOVED
FALCON 9 DEB, 2020-055BM,  1.187062847177365 REMOVED
COSMOS 1408 DEB, 1982-092RY,  1.3262911429437225 REMOVED
C

In [17]:
final_t = 365.25 * pk.DAY2SEC

pbar = tqdm(total=final_t)

while sim.time < final_t:
    orig_time = sim.time
    
    oc = sim.step()
    
    pbar.update(sim.time - orig_time)
   
    if oc == csc.outcome.collision:
        # TODO different code needed for crash
        # on Earth here.
        pi, pj = sim.interrupt_info
        
        print("Collision detected, re-initing particles {} and {}".format(pi, pj))
        
        for idx in [pi, pj]:
            a = rng.uniform(1.02*Re, 1.3*Re)
            e = rng.uniform(0, 0.02)
            inc = rng.uniform(0, 0.05)
            om = rng.uniform(0, 2*np.pi)
            Om = rng.uniform(0, 2*np.pi)
            nu = rng.uniform(0, 2*np.pi)
            size = rng.uniform(0.01, 0.1)

            r, v = pk.par2ic([a, e, inc, om, Om, nu], pk.MU_EARTH)

            sim.state[idx,0:3] = r
            sim.state[idx,3:6] = v
            sim.state[idx,6] = size
    elif oc == csc.outcome.reentry:
        pi = sim.interrupt_info
        # We log on screen 
        print(satcat[to_satcat[pi]]["OBJECT_NAME"].strip() + ", " + satcat[to_satcat[pi]]["OBJECT_ID"].strip() + ", ", sim.time*pk.SEC2DAY, "REMOVED")
        # We remove the re-entered object and restart the simulation
        new_state = np.delete(sim.state,pi,axis=0)
        new_pars = np.delete(sim.pars,pi,axis=0)
        
        sim.set_new_state(new_state)
        sim.pars[:] = new_pars
        
        # new_r_ic = np.vstack((sim.x,sim.y,sim.z)).transpose()
        # new_v_ic = np.vstack((sim.vx,sim.vy,sim.vz)).transpose()
        # new_r_ic, new_v_ic, new_BSTARS,new_to_satcat, new_c_radius = remove_particle(pi, new_r_ic, new_v_ic, new_BSTARS,new_to_satcat, new_c_radius)
        # sim.set_new_state(new_r_ic[:,0],new_r_ic[:,1],new_r_ic[:,2],new_v_ic[:,0],new_v_ic[:,1],new_v_ic[:,2],new_c_radius, pars=[new_BSTARS])
pbar.close()
del pbar

  0%|          | 0/31557600.0 [00:00<?, ?it/s]

COSMOS 1408 DEB, 1982-092FH,  0.001101804996265661 REMOVED
COSMOS 1408 DEB, 1982-092GL,  0.004102843891965073 REMOVED
SL-4 R/B, 2006-061B,  0.005706346736237456 REMOVED
FREGAT DEB, 2011-037EQ,  0.007267201428668006 REMOVED
STARLINK-2025, 2021-009BM,  0.018153158990584835 REMOVED
COSMOS 1408 DEB, 1982-092ADW,  0.029528243104741347 REMOVED
COSMOS 2241, 1993-022A,  0.04331453507422833 REMOVED
OBJECT K, 2021-002K,  0.06080187246401055 REMOVED
RESURS O1 DEB, 1994-074BA,  0.07344824309334752 REMOVED
FREGAT DEB, 2011-037NN,  0.0789510521099178 REMOVED
COSMOS 1408 DEB, 1982-092TW,  0.07895352180902593 REMOVED
COSMOS 1408 DEB, 1982-092AHA,  0.11577295604283491 REMOVED
STARLINK-1906, 2020-074AC,  0.15794831123395808 REMOVED
OBJECT B, 2021-117B,  0.5273460439998182 REMOVED
COSMOS 2251 DEB, 1993-036AEX,  0.6732365054153558 REMOVED
COSMOS 1408 DEB, 1982-092FX,  1.128301603795796 REMOVED
FALCON 9 DEB, 2020-055BM,  1.1870635028135692 REMOVED
COSMOS 1408 DEB, 1982-092RY,  1.3262372568571101 REMOVED
CO

KeyboardInterrupt: 